In [1]:
import codecs
from dataclasses import dataclass
from typing import List, Dict, Any

import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch import Tensor
from torch import nn
import time

import random
random.seed(0)
torch.manual_seed(0)

# from transformers import BertTokenizer, BertModel
# bert_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
# bert_model = BertModel.from_pretrained("bert-base-cased")
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '/work/nlp-project')
from scripts.read_write_data import read_processed_data, write_baseline_pred
from scripts.evaluation_functions import f1_score, tag_accuracy
from models.classes import DataIterator, Batch

# import gensim.models
# GoogleEmbs = gensim.models.KeyedVectors.load_word2vec_format(
#                                 '/work/nlp-project/models/GoogleNews-50k.bin', binary=True)

import matplotlib.pyplot as plt

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
TRAIN_PATH = "nlp-project/data/processed/train_splits/labeled.conll"

# Loading data

x_train = []
y_train = []
for words, labels, _, _ in read_processed_data(TRAIN_PATH):
    x_train.append(words)
    y_train.append(labels)


In [3]:
x_train[:10]

[['Lol', '.'],
 ['Jim'],
 ['Who',
  'could',
  'stop',
  'a',
  'suicide',
  'murder',
  'in',
  'the',
  'midst',
  'of',
  'the',
  'crowded',
  'line',
  'waiting',
  'to',
  'be',
  'checked',
  'by',
  'the',
  'airport',
  'metal',
  'detector',
  '?'],
 ['With',
  'going',
  'outside',
  ',',
  'put',
  'the',
  'leash',
  'on',
  'and',
  'just',
  'take',
  'him',
  'for',
  'a',
  'walk',
  '.'],
 ['I',
  'have',
  'many',
  'friends',
  'that',
  'have',
  'laptops',
  'bought',
  'in',
  'the',
  'UK',
  'that',
  'use',
  'them',
  'here'],
 ['10/23/2000', '12:12', 'PM'],
 ['It', 'is', 'short', 'for', 'San', 'Francisco', '.'],
 ['What',
  'do',
  'the',
  'new',
  'al',
  '-',
  'Qaeda',
  'videotape',
  'and',
  'audio',
  'speeches',
  'of',
  'Bin',
  'Laden',
  'and',
  'Ayman',
  'al',
  '-',
  'Zawahiri',
  'tell',
  'us',
  'about',
  'the',
  'hopes',
  'of',
  'the',
  'remaining',
  'top',
  'leadership',
  'of',
  'the',
  'organization',
  '?'],
 ['11/29/2000',

In [5]:
ppdb_file=pd.read_csv('/work/nlp-project/models/ppdb-2.0-s-lexical.csv')
ppdb_file

,typeofw,word1,word2,score
0,[NN],transplant,transplantation,5.24981
1,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,5.22098
2,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,5.21828
3,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,5.21462
4,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,5.21153
...,...,...,...,...
231711,[NNS],examinations,monitors,3.47444
231712,[VBZ],focuses,essentially,3.47444
231713,[VBN],acquired,acceded,3.47444
231714,[VBN],debated,regulated,3.47444


In [6]:
ppdb_file = ppdb_file.drop_duplicates(subset=['word1', 'word2'])

In [7]:
ppdb_file['word1'] = ppdb_file['word1'].astype(str).str.strip().str.lower()
ppdb_file['word2'] = ppdb_file['word2'].astype(str).str.strip().str.lower()


/tmp/ipykernel_238/2112911357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppdb_file['word1'] = ppdb_file['word1'].astype(str).str.strip().str.lower()
/tmp/ipykernel_238/2112911357.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ppdb_file['word2'] = ppdb_file['word2'].astype(str).str.strip().str.lower()


In [8]:

def generate_paraphrases(sentence, ppdb_file, max_paraphrases=10):
    paraphrases = set()
    for index, row in ppdb_file.iterrows():
        if row['word1'] in sentence:
            paraphrases.add(sentence.replace(row['word1'], row['word2']))
        elif row['word2'] in sentence:
            paraphrases.add(sentence.replace(row['word2'], row['word1']))
        if len(paraphrases) >= max_paraphrases:
            break
    return list(paraphrases)


In [9]:
sentence = "I work on the project."
paraphrased_sentences = generate_paraphrases(sentence, ppdb_file)
print(paraphrased_sentences)

['I work on the programmes.', 'I workloads on the project.', 'I work on the proposes.', 'I workplaces on the project.', 'I work on the professionalsject.', 'I working on the project.', 'I work on the projects.', 'I workers on the project.', 'I work on the planned.', 'I endeavours on the project.']


In [11]:
TRAIN_PATH = "nlp-project/data/processed/train.conll"
DEV_PATH = "nlp-project/data/processed/dev.conll"
TEST_PATH = "nlp-project/data/processed/test.conll"

# Loading data

x_train = []
y_train = []
for words, labels, _, _ in read_processed_data(TRAIN_PATH):
    x_train.append(words)
    y_train.append(labels)

x_dev = []
y_dev = []
for words, labels, _, _ in read_processed_data(DEV_PATH):
    x_dev.append(words)
    y_dev.append(labels)

In [12]:
ppdb_file.head()

,typeofw,word1,word2,score
0,[NN],transplant,transplantation,5.24981
1,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,5.22098
2,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,5.21828
3,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,5.21462
4,[JJ],<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,<www.un.org/depts/dgacm/docs/crp/aconf212crp1/...,5.21153


In [13]:
def paraphrase_word(word):
    for w1, w2 in zip(ppdb_file['word1'], ppdb_file['word2']):
        if word.casefold() == w1.casefold():
            return w2
        elif word.casefold() == w2.casefold():
            return w1
    return None

In [0]:
# def paraphrase_word_options(word):
#     options = []
#     scores = []
#     for w1, w2, score in zip(ppdb_file['word1'], ppdb_file['word2'], ppdb_file['score']):
#         if word.casefold() == w1.casefold():
#             options.append(w2)
#             scores.append(score)
#         elif word.casefold() == w2.casefold():
#             options.append(w1)
#             scores.append(score)

#     return options, scores
        

In [14]:
NEs= []
for sentence, labels in zip(x_train,y_train):
    for word, label in zip(sentence, labels):
        if label == '1':
            NEs.append(word)

paraphrases = []
for NE in NEs:
    paraphrase = paraphrase_word(NE)
    paraphrases.append(paraphrase)
    # print(NE, paraphrase)
print(f"Proportion of NEs with no paraphrase: {np.mean(np.array(paraphrases) == None)}")



Proportion of NEs with no paraphrase: 0.6265221369747308


Proportion of NEs with no paraphrase: 0.629

In [15]:
for NE, p in zip(NEs, paraphrases):
    if p != '--------':
        print(NE, p)

Saddam None
Arafat None
Bin ibn
Laden None
Israeli israelis
Arab arabic
European europe
Orwellian None
Western west
Joseph None
Goebbels None
Iran iranian
Syria syrians
Iraq iraqi
Libya libyan
Palestinian palestinians
Saudi saudis
Arabia None
United unity
States stated
Europe european
European europe
United unity
Nations nation-states
Saudi saudis
Saudis saudi
Arab arabic
Europe european
Iraq iraqi
Switzerland None
Arafat None
Paris paris-based
Palestinian palestinians
Al None
- ls
Aksa None
Arafat None
dollars u.s.dollars
Iraq iraqi
Palestinian palestinians
Teheran tehran
Iranian iran
Spain None
France None
Iran iranian
Spain None
France None
Western west
Western west
Europe european
American america
- ls
led co-lead
Iraq iraqi
Western west
Asia asian
Afghanistan afghan
Iraq iraqi
Libya libyan
Iran iranian
Syria syrians
Lebanon None
Syrian syria
Sudan sudanese
Afghanistan afghan
Iraq iraqi
Iran iranian
Syria syrians
Iran iranian
Afghanistan afghan
Gulf None
States stated
Iraq iraqi
So

Building paraphrased sentences:

In [0]:
p_train = []
for sentence in x_train:
    p_sentence = []
    for word in sentence:
        p_word = paraphrase_word(word)
        if p_word is None:
            p_sentence.append(word)
        else:
            p_sentence.append(p_word)
    p_train.append(p_sentence)

KeyboardInterrupt: 

In [1]:
p_train

NameError: name 'p_train' is not defined

In [0]:
np.array(p_train).shape, np.array(y_train).shape

In [0]:
data = pd.DataFrame([p_train,y_train]).transpose()
# write_baseline_pred(data, "/work/nlp-project/data/paraphrased/train_labeled.conll")

In [ ]:
# TRAIN_PATH = "nlp-project/data/processed/train.conll"
TRAIN_PATH = "/work/nlp-project/data/processed/train_splits/labeled.conll"
DEV_PATH = "nlp-project/data/processed/dev.conll"

# Loading data
x_train, y_train, _, _ = load_data(TRAIN_PATH)
x_dev, y_dev, _, _ = load_data(DEV_PATH)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b2f14aee-af04-4db5-af55-57a3a58b9f40' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>